#### **Ingesta de la carpeta "movie_languages"**

In [0]:
dbutils.widgets.text("environment","")
var_environment = dbutils.widgets.get("environment")

In [0]:
dbutils.widgets.text("file_date","2024-12-30")
var_file_date = dbutils.widgets.get("file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

##### Librerías

In [0]:
from pyspark.sql.functions import lit,current_timestamp
from pyspark.sql.types import StructType,StructField,StringType,IntegerType

##### Esquema

In [0]:
movie_languages_schema = StructType([
    StructField("movieId",IntegerType(),False),
    StructField("languageId",IntegerType(),False),
    StructField("languageRoleId",IntegerType(),False)
])

##### Leer ficheros JSON

In [0]:
df_movie_languages = spark.read.option("multiline", True).schema(movie_languages_schema).json(f"{bronze_folder_path}/{var_file_date}/movie_language")

##### Renombrar y añadir columnas

In [0]:
df_movie_languages_final = add_ingestion_date(df_movie_languages).withColumnsRenamed({"movieId":"movie_id","languageId":"language_id"}) \
                                             .withColumn("environment",lit(var_environment)).drop("languageRoleId").withColumn("file_date",lit(var_file_date))

##### Escribir en Silver

In [0]:
merge_condition = "target.movie_id = source.movie_id AND target.language_id = source.language_id AND target.file_date = source.file_date"
merge_delta_lake(df_movie_languages_final,"movie_silver","movies_languages",silver_folder_path,merge_condition,"file_date")

In [0]:
%sql
SELECT COUNT(1), file_date
FROM movie_silver.movies_languages
GROUP BY file_date

In [0]:
dbutils.notebook.exit("Success")